In [1]:
import sys
# !{sys.executable} -m pip install git+git://github.com/EnzymeML/PyEnzyme.git@Refactoring

# Modeling a reaction by mass action cascades

This notebook is part of the publication "EnzymeML at Work" from Lauterbach et al. 2022 and compares the fitting of a micro-kinetic model (specified in an EnzymeML document) to experimental data (specified in the same EnzymeML document). 

Generation of the EnzymeML document and individual fitting of the data with either PySCeS or COPASI have been dealt with in separate notebooks.

-----
## Kinetic Modeling

Now that the EnzymeMLDocument has been adapted to the micro-kinetic model, it can be modeled and optimized using PySCeS and COPASI. Since both modeling package interfaces are an integral part of PyEnzyme, called Thin Layer, a simple call to the corresponding Thin Layer object is necessary.

But before optimization, it might be necessary to define initial values. Since manipulating the KineticParameter initial_values attributes inside the script that generates the EnzymeMLDocument can get quite tedious, PyEnzyme offers an external data structure from within initial values can be applied. This way, the EnzymeML document is only modifed at optimization and remains untouched until then.

The initialization file is in the YAML format and contains all reactions and their parameters. In addtion, PyEnzyme offers a method to generate such a YAML file, which can be edited manually.

In [2]:
# Generate a blank YAML file to manually enter initial values for modeling
from pyenzyme import EnzymeMLDocument

enzmldoc = EnzymeMLDocument.fromFile("Model_4.omex")
enzmldoc.generateInitialValueTemplate()

In [3]:
from pyenzyme.thinlayers import ThinLayerPysces

Matplotlib backend set to: "nbAgg"
Matplotlib interface loaded (pysces.plt.m)
Pitcon routines available
NLEQ2 routines available
SBML support available
You are using NumPy (1.21.2) with SciPy (1.7.0)
RateChar is available

No module named 'ipyparallel'
INFO: Parallel scanner not available

PySCeS environment
******************
pysces.model_dir = /Users/janrange/Pysces/psc
pysces.output_dir = /Users/janrange/Pysces


***********************************************************************
* Welcome to PySCeS (1.0.1) - Python Simulator for Cellular Systems   *
*                http://pysces.sourceforge.net                        *
* Copyright(C) B.G. Olivier, J.M. Rohwer, J.-H.S. Hofmeyr, 2004-2022  *
* Triple-J Group for Molecular Cell Physiology                        *
* Stellenbosch University, ZA and VU University Amsterdam, NL         *
* PySCeS is distributed under the PySCeS (BSD style) licence, see     *
* LICENCE.txt (supplied with this release) for details                *
* Pl

In [4]:
# Initialize the layer
tl_pysces = ThinLayerPysces(
    "Model_4.omex", 
    init_file="EnzymeML_Lagerman_init_copasi.yaml",
    model_dir="pySCeS"
)

Check SBML support is at action level 2
SBML file is L3V2



*********ERRORS***********


*********ERRORS***********


Possible errors detected in SBML conversion, Model may be incomplete. Please check the error log file "EnzymeML_Lagerman.xml-sbml_conversion_errors.txt" for details.


*******************************************************************
Issues encountered in SBML translation (model processed anyway)
SBML source: pySCeS/EnzymeML_Lagerman.xml
*******************************************************************

Parameter units ignored for parameters:
['K_si', 'K_n'] 

Parameter units ignored for (local) parameters:
['v_r', 'K_s', 'k_2', 'k_6', 'k_3', 'K_pg', 'k_5', 'k_4', 'k_4b', 'K_p', 'k_d'] 

*******************************************************************

Info: single compartment model: locating "r0" in default compartment
Info: single compartment model: locating "r1" in default compartment
Info: single compartment model: locating "r2" in default compartment
Info: 

In [5]:
# Run optimization
tl_pysces.model.__settings__["lsoda_mxstep"] = 3000
tl_pysces.optimize(method='least_squares')

# Write to new EnzymeMLDocument and save
pysces_doc = tl_pysces.write()
pysces_doc.toFile(".", name="EnzymeML_Lagerman_M4_PySCeS_Modeled")


Archive was written to ./EnzymeML_Lagerman_M4_PySCeS_Modeled.omex



## Using the COPASI thin layer

In the same manner the COPASI Thin Layer can be used to model the given data. Thin Layers require to follow a given metaclass and thus the syntax of every modeling layer follows the Initialization > ```optimize```-method > ```write```-method procedure. 

The COPASI optimization is set up to use the same initial values and the same fitting algorithm that was used with PySCeS, to allow an easy comparison.

In [6]:
from basico import set_current_model, set_task_settings, T, PE
from pyenzyme.thinlayers import ThinLayerCopasi

In [7]:
# Initialize COPASI Thin Layer
tl_copasi = ThinLayerCopasi(
    "Model_4.omex", "COPASI",
    init_file="EnzymeML_Lagerman_init_copasi.yaml"
)

set_current_model(tl_copasi.dm)
set_task_settings(T.PARAMETER_ESTIMATION, 
                  {
                      'scheduled': True,
                      'problem': {'Randomize Start Values': False},
                      'method': {'name': PE.LEVENBERG_MARQUARDT}
                  })

tl_copasi.optimize()

In [8]:
copasi_doc = tl_copasi.write()
copasi_doc.toFile(".", name="EnzymeML_Lagerman_M4_COPASI_Modeled")

DEBUG:pyenzyme:KineticParameter 'K_si' - value was set from '16.12556695' to '16.114987233151997'
DEBUG:pyenzyme:KineticParameter 'K_n' - value was set from '14.16013538' to '14.168483907055077'
DEBUG:pyenzyme:KineticParameter 'K_s' - value was set from '0.9517666749' to '0.9507535476900487'
DEBUG:pyenzyme:KineticParameter 'k_2' - value was set from '32894.518' to '32862.95620531188'
DEBUG:pyenzyme:KineticParameter 'k_6' - value was set from '1364.827977' to '1364.4033354617336'
DEBUG:pyenzyme:KineticParameter 'k_3' - value was set from '999911.3271' to '999219.6373831154'
DEBUG:pyenzyme:KineticParameter 'K_pg' - value was set from '163.4985656' to '163.46716926999855'
DEBUG:pyenzyme:KineticParameter 'k_5' - value was set from '438.2508805' to '438.2888429562448'
DEBUG:pyenzyme:KineticParameter 'k_4' - value was set from '13120.92495' to '13114.702442929261'
DEBUG:pyenzyme:KineticParameter 'k_4b' - value was set from '572987.5006' to '572414.2215109317'
DEBUG:pyenzyme:KineticParameter 


Archive was written to ./EnzymeML_Lagerman_M4_COPASI_Modeled.omex



Both results can now be compared by individually exporting the estimated parameters using the ```exportKineticParameters```-method found in the ```EnzymeMLDocument``` instance that returns a Pandas ```DataFrame``` object. Finally, for the sake of comparison, both result are merged in a single ```DataFrame```.

In [9]:
params = pysces_doc.exportKineticParameters(exclude_constant=True)
params.rename({"value": "PySCeS"}, axis="columns", inplace=True)
params["COPASI"] = copasi_doc.exportKineticParameters(exclude_constant=True).value

params[["name", "PySCeS", "COPASI", "unit"]]

,name,PySCeS,COPASI,unit
reaction,,,,
r1,K_s,0.948655,0.950754,l / mmole
r2,k_2,30182.540153,32862.956205,1 / s
r5,k_6,1414.817976,1364.403335,1 / s
r6,k_3,999840.708222,999219.637383,1 / s
r7,K_pg,160.618723,163.467169,l / mmole
r9,k_5,439.674702,438.288843,1 / s
r10,k_4,13588.682715,13114.702443,1 / s
r10,k_4b,557559.494809,572414.221511,1 / s
r11,K_p,99.963582,102.181182,mmole / l


-------